In [1]:
import os
%matplotlib inline
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import geoviews as gv

Load Shapefile of Toronto Municipal Wards

In [2]:
shp_file = './Toronto/wards_may2010_wgs84/gcc/Projects/Open Data/Files/Data Upload - May 2010/May2010_WGS84'
toronto2010 = gpd.read_file(shp_file)

Ward 1 Mayoral Election. Each Ward is on its own sheet and I am currently only interested in the totals so I will create a new dataframe witht the total from each ward. 

In [3]:
excelmayor2010 = './Toronto/2010_results/2010_Toronto_Poll_by_Poll_Mayor.xls'
mayor10 = pd.DataFrame()
mayor10["Candidates"] = pd.read_excel(io = excelmayor2010, sheet_name = 1, header=1)["Subdivision"]

Populate the pandas dataframe to use with shape files

Extract Total Votes from each Toronto City Ward

In [4]:
for i in range(44):
    mayor2010 = pd.read_excel(io = excelmayor2010, sheet_name = i, header=1)
    mayor10[str(i+1)] = mayor2010["Total"].astype("int")
mayor10.head()

,Candidates,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,ACHAMPONG ROCCO,151,52,39,37,45,57,123,209,98,...,51,44,40,72,38,74,51,112,69,56
1,ALEXANDER MICHAEL,49,28,20,28,49,56,77,163,83,...,118,34,102,76,76,106,54,117,69,42
2,ANDREWS DON,26,18,10,8,17,14,31,47,27,...,47,22,26,31,30,39,39,42,27,33
3,BABULA GEORGE,90,25,43,50,47,58,88,101,64,...,106,54,109,117,76,109,94,319,114,68
4,BALL CHRISTOPHER,21,12,11,10,17,8,28,27,18,...,18,9,16,25,19,30,22,43,31,16


In [5]:
mayor10.Candidates = mayor10.Candidates.astype("str")
mayor10.Candidates = mayor10.Candidates.replace(' ', '_', regex=True)
mayor10f = mayor10.drop(index=40)

In [6]:
mayor10f = mayor10f.T.reset_index()
mayor10f.columns = mayor10f.iloc[0]
mayor10f = mayor10f.drop(index=0)
mayor10f = mayor10f.reset_index()
mayor10f = mayor10f.rename(columns = {"Candidates": "SCODE_NAME"})
mayor10f.SCODE_NAME = mayor10f.SCODE_NAME.astype("int")
toronto2010.SCODE_NAME = toronto2010.SCODE_NAME.astype('int')
mayor10f = mayor10f.astype("int")

In [7]:
to_mayor_election = gpd.GeoDataFrame(pd.merge(toronto2010, mayor10f))

## Write Clean File

In [8]:
os.chdir("./Cleaned Data")
file = open("2010_Toronto_Mayoral_Election.json", "w+")
file.write(to_mayor_election.to_json())
file.close

<function TextIOWrapper.close()>